In [1]:
file_dir = "../ariel_results/per_function_timing/"

import os, re, pandas as pd

TIMING_RE = re.compile(
    r"^\s*(?:-\s*)*"                 # leading “- ” blocks
    r"(?P<name>[^:]+?)\s+"           # function name
    r"(?:Took|took):\s+"             # “took:” (any case)
    r"(?P<secs>[0-9.eE+-]+)s",
    re.IGNORECASE,
)

def _parse(log: str) -> pd.DataFrame:
    rows, cur_tree = [], -1
    for line in log.splitlines():
        if "Selecting Bootstrapped Samples Took" in line:
            cur_tree += 1
        m = TIMING_RE.match(line)
        if m and cur_tree >= 0:
            rows.append(
                {
                    "tree": cur_tree + 1,
                    "function": m.group("name").strip(),
                    "time_s": float(m.group("secs")),
                }
            )
    return pd.DataFrame(rows)

# ── NEW: desired column order & renames ────────────────────────────────
ORDER = [
    "Selecting Bootstrapped Samples",
    "Initialization of FindBestCondOblique",
    "SampleProjection",
    "ApplyProjection",
    "Bucket Allocation & Initialization=0",
    "Filling & Finalizing the Buckets",
    "SortFeature",
    "ScanSplits",
    "Post-processing after Training all Trees",
    "EvaluateProjection",
    "FillExampleBucketSet (next 3 calls)",
]
RENAMES = {
    "Post-processing after Train": "Post-processing after Training all Trees",
    "FillExampleBucketSet (calls 3 above)": "FillExampleBucketSet (next 3 calls)",
}

def tree_table(log: str) -> pd.DataFrame:
    df = _parse(log)
    long = df.groupby(["tree", "function"], as_index=False)["time_s"].sum()

    wide = (
        long.pivot(index="tree", columns="function", values="time_s")
            .rename(columns=RENAMES)          # make names match ORDER list
            .fillna(0.0)
    )

    # ensure every requested column exists, then apply the order
    wide = wide.reindex(columns=ORDER, fill_value=0.0)

    # make “tree” a visible column instead of the index
    wide = wide.reset_index()

    return wide


In [21]:
raw_results = """
Selecting Bootstrapped Samples Took: 0.0549813s

 - Initialization of FindBestCondOblique Took: 0.00249373s


 - SampleProjection took: 8.863e-06s
 - ApplyProjection took: 0.00339117s

 - - Bucket Allocation & Initialization=0 took: 0.00184835s
 - - Filling & Finalizing the Buckets took: 3.7e-08s
 - - SortFeature took: 0.0527089s
 - FillExampleBucketSet (calls 3 above) took: 0.0552279s

 - ScanSplits took: 0.010009s

 - EvaluateProjection took: 0.0655519s



DecisionTree::Train() alone Took: 0.0835021s

I0000 00:00:1749576743.753999   13281 random_forest.cc:980] Train tree 1/20 [index:0 total:0.14s tree:0.14s]

Selecting Bootstrapped Samples Took: 0.0550472s

 - Initialization of FindBestCondOblique Took: 0.00236495s


 - SampleProjection took: 3.406e-06s
 - ApplyProjection took: 0.00340789s

 - - Bucket Allocation & Initialization=0 took: 0.00183488s
 - - Filling & Finalizing the Buckets took: 3.8e-08s
 - - SortFeature took: 0.0550827s
 - FillExampleBucketSet (calls 3 above) took: 0.0575992s

 - ScanSplits took: 0.0100198s

 - EvaluateProjection took: 0.0679332s



DecisionTree::Train() alone Took: 0.0829249s


Selecting Bootstrapped Samples Took: 0.0536879s

 - Initialization of FindBestCondOblique Took: 0.00234877s


 - SampleProjection took: 2.859e-06s
 - ApplyProjection took: 0.00340315s

 - - Bucket Allocation & Initialization=0 took: 0.00184253s
 - - Filling & Finalizing the Buckets took: 3.7e-08s
 - - SortFeature took: 0.0580991s
 - FillExampleBucketSet (calls 3 above) took: 0.0606234s

 - ScanSplits took: 0.0097867s

 - EvaluateProjection took: 0.0707228s



DecisionTree::Train() alone Took: 0.0848728s


Selecting Bootstrapped Samples Took: 0.0523445s

 - Initialization of FindBestCondOblique Took: 0.00234366s


 - SampleProjection took: 2.755e-06s
 - ApplyProjection took: 0.00339119s

 - - Bucket Allocation & Initialization=0 took: 0.00180412s
 - - Filling & Finalizing the Buckets took: 2.6e-08s
 - - SortFeature took: 0.0532206s
 - FillExampleBucketSet (calls 3 above) took: 0.0557205s

 - ScanSplits took: 0.00976742s

 - EvaluateProjection took: 0.0658s



DecisionTree::Train() alone Took: 0.0798562s


Selecting Bootstrapped Samples Took: 0.0523999s

 - Initialization of FindBestCondOblique Took: 0.00285539s


 - SampleProjection took: 5.536e-06s
 - ApplyProjection took: 0.00350885s

 - - Bucket Allocation & Initialization=0 took: 0.00179905s
 - - Filling & Finalizing the Buckets took: 2.6e-08s
 - - SortFeature took: 0.0530813s
 - FillExampleBucketSet (calls 3 above) took: 0.0555543s

 - ScanSplits took: 0.00981525s

 - EvaluateProjection took: 0.0656824s



DecisionTree::Train() alone Took: 0.0847869s


Selecting Bootstrapped Samples Took: 0.0523906s

 - Initialization of FindBestCondOblique Took: 0.00241779s


 - SampleProjection took: 4.761e-06s
 - ApplyProjection took: 0.00340702s

 - - Bucket Allocation & Initialization=0 took: 0.0018065s
 - - Filling & Finalizing the Buckets took: 2.6e-08s
 - - SortFeature took: 0.0534887s
 - FillExampleBucketSet (calls 3 above) took: 0.0559934s

 - ScanSplits took: 0.00977505s

 - EvaluateProjection took: 0.0660846s



DecisionTree::Train() alone Took: 0.0809695s


Selecting Bootstrapped Samples Took: 0.0532917s

 - Initialization of FindBestCondOblique Took: 0.00278442s


 - SampleProjection took: 6.34e-06s
 - ApplyProjection took: 0.00340527s

 - - Bucket Allocation & Initialization=0 took: 0.00183381s
 - - Filling & Finalizing the Buckets took: 2.7e-08s
 - - SortFeature took: 0.0534703s
 - FillExampleBucketSet (calls 3 above) took: 0.0559578s

 - ScanSplits took: 0.0097914s

 - EvaluateProjection took: 0.0660618s



DecisionTree::Train() alone Took: 0.080687s


Selecting Bootstrapped Samples Took: 0.0525063s

 - Initialization of FindBestCondOblique Took: 0.00233399s


 - SampleProjection took: 4.746e-06s
 - ApplyProjection took: 0.00340113s

 - - Bucket Allocation & Initialization=0 took: 0.00185546s
 - - Filling & Finalizing the Buckets took: 2.6e-08s
 - - SortFeature took: 0.0546784s
 - FillExampleBucketSet (calls 3 above) took: 0.0572328s

 - ScanSplits took: 0.00976247s

 - EvaluateProjection took: 0.0673106s



DecisionTree::Train() alone Took: 0.0811506s


Selecting Bootstrapped Samples Took: 0.0523344s

 - Initialization of FindBestCondOblique Took: 0.00230953s


 - SampleProjection took: 4.937e-06s
 - ApplyProjection took: 0.00338712s

 - - Bucket Allocation & Initialization=0 took: 0.00185486s
 - - Filling & Finalizing the Buckets took: 2.6e-08s
 - - SortFeature took: 0.054761s
 - FillExampleBucketSet (calls 3 above) took: 0.0572771s

 - ScanSplits took: 0.0097777s

 - EvaluateProjection took: 0.067366s



DecisionTree::Train() alone Took: 0.0805491s


Selecting Bootstrapped Samples Took: 0.0532496s

 - Initialization of FindBestCondOblique Took: 0.00232934s


 - SampleProjection took: 6.052e-06s
 - ApplyProjection took: 0.00339423s

 - - Bucket Allocation & Initialization=0 took: 0.0018384s
 - - Filling & Finalizing the Buckets took: 2.5e-08s
 - - SortFeature took: 0.0533874s
 - FillExampleBucketSet (calls 3 above) took: 0.0559511s

 - ScanSplits took: 0.00985549s

 - EvaluateProjection took: 0.0661178s



DecisionTree::Train() alone Took: 0.083938s


Selecting Bootstrapped Samples Took: 0.0525328s

 - Initialization of FindBestCondOblique Took: 0.00216895s


 - SampleProjection took: 4.283e-06s
 - ApplyProjection took: 0.00334687s

 - - Bucket Allocation & Initialization=0 took: 0.00184122s
 - - Filling & Finalizing the Buckets took: 2.7e-08s
 - - SortFeature took: 0.115961s
 - FillExampleBucketSet (calls 3 above) took: 0.118498s

 - ScanSplits took: 0.0304544s

 - EvaluateProjection took: 0.149319s



DecisionTree::Train() alone Took: 0.16608s


Selecting Bootstrapped Samples Took: 0.0555125s

 - Initialization of FindBestCondOblique Took: 0.00233943s


 - SampleProjection took: 5.78e-06s
 - ApplyProjection took: 0.00353316s

 - - Bucket Allocation & Initialization=0 took: 0.00200933s
 - - Filling & Finalizing the Buckets took: 3.7e-08s
 - - SortFeature took: 0.0545373s
 - FillExampleBucketSet (calls 3 above) took: 0.057247s

 - ScanSplits took: 0.00998054s

 - EvaluateProjection took: 0.0675389s



DecisionTree::Train() alone Took: 0.0817234s


Selecting Bootstrapped Samples Took: 0.122805s

 - Initialization of FindBestCondOblique Took: 0.00446599s


 - SampleProjection took: 1.3589e-05s
 - ApplyProjection took: 0.00346861s

 - - Bucket Allocation & Initialization=0 took: 0.00187815s
 - - Filling & Finalizing the Buckets took: 3.6e-08s
 - - SortFeature took: 0.0551487s
 - FillExampleBucketSet (calls 3 above) took: 0.0577108s

 - ScanSplits took: 0.00988154s

 - EvaluateProjection took: 0.0679828s



DecisionTree::Train() alone Took: 0.0876307s


Selecting Bootstrapped Samples Took: 0.0546206s

 - Initialization of FindBestCondOblique Took: 0.00232899s


 - SampleProjection took: 5.761e-06s
 - ApplyProjection took: 0.00335943s

 - - Bucket Allocation & Initialization=0 took: 0.00185351s
 - - Filling & Finalizing the Buckets took: 2.6e-08s
 - - SortFeature took: 0.0548512s
 - FillExampleBucketSet (calls 3 above) took: 0.0574576s

 - ScanSplits took: 0.00985751s

 - EvaluateProjection took: 0.0676307s



DecisionTree::Train() alone Took: 0.0846132s


Selecting Bootstrapped Samples Took: 0.0755949s

 - Initialization of FindBestCondOblique Took: 0.0063153s


 - SampleProjection took: 3.676e-05s
 - ApplyProjection took: 0.00624438s

 - - Bucket Allocation & Initialization=0 took: 0.00185772s
 - - Filling & Finalizing the Buckets took: 3.7e-08s
 - - SortFeature took: 0.0549215s
 - FillExampleBucketSet (calls 3 above) took: 0.0575028s

 - ScanSplits took: 0.0098914s

 - EvaluateProjection took: 0.0677286s



DecisionTree::Train() alone Took: 0.0913487s


Selecting Bootstrapped Samples Took: 0.0536581s

 - Initialization of FindBestCondOblique Took: 0.00228594s


 - SampleProjection took: 5.694e-06s
 - ApplyProjection took: 0.00334618s

 - - Bucket Allocation & Initialization=0 took: 0.001832s
 - - Filling & Finalizing the Buckets took: 2.6e-08s
 - - SortFeature took: 0.0549543s
 - FillExampleBucketSet (calls 3 above) took: 0.0575927s

 - ScanSplits took: 0.00983681s

 - EvaluateProjection took: 0.0677457s



DecisionTree::Train() alone Took: 0.0853607s


Selecting Bootstrapped Samples Took: 0.0549007s

 - Initialization of FindBestCondOblique Took: 0.00249315s


 - SampleProjection took: 5.778e-06s
 - ApplyProjection took: 0.00346658s

 - - Bucket Allocation & Initialization=0 took: 0.00181879s
 - - Filling & Finalizing the Buckets took: 2.6e-08s
 - - SortFeature took: 0.0549365s
 - FillExampleBucketSet (calls 3 above) took: 0.0574893s

 - ScanSplits took: 0.0098152s

 - EvaluateProjection took: 0.067617s



DecisionTree::Train() alone Took: 0.0806729s


Selecting Bootstrapped Samples Took: 0.0550875s

 - Initialization of FindBestCondOblique Took: 0.00284359s


 - SampleProjection took: 7.405e-06s
 - ApplyProjection took: 0.00389405s

 - - Bucket Allocation & Initialization=0 took: 0.00181051s
 - - Filling & Finalizing the Buckets took: 2.6e-08s
 - - SortFeature took: 0.110562s
 - FillExampleBucketSet (calls 3 above) took: 0.113109s

 - ScanSplits took: 0.0300753s

 - EvaluateProjection took: 0.143555s



DecisionTree::Train() alone Took: 0.166924s


Selecting Bootstrapped Samples Took: 0.131706s

 - Initialization of FindBestCondOblique Took: 0.00495856s


 - SampleProjection took: 1.6612e-05s
 - ApplyProjection took: 0.00817686s

 - - Bucket Allocation & Initialization=0 took: 0.00359666s
 - - Filling & Finalizing the Buckets took: 1.01e-07s
 - - SortFeature took: 0.128782s
 - FillExampleBucketSet (calls 3 above) took: 0.133826s

 - ScanSplits took: 0.0300417s

 - EvaluateProjection took: 0.164501s



DecisionTree::Train() alone Took: 0.197382s


Selecting Bootstrapped Samples Took: 0.131574s

 - Initialization of FindBestCondOblique Took: 0.0043103s


 - SampleProjection took: 8.635e-06s
 - ApplyProjection took: 0.00393927s

 - - Bucket Allocation & Initialization=0 took: 0.00184563s
 - - Filling & Finalizing the Buckets took: 3.8e-08s
 - - SortFeature took: 0.0549458s
 - FillExampleBucketSet (calls 3 above) took: 0.0574921s

 - ScanSplits took: 0.00984937s

 - EvaluateProjection took: 0.067657s



DecisionTree::Train() alone Took: 0.0863477s


Post-processing after Train Took: 0.000405017s

Training wall-time: 3.27886s
"""

In [22]:
tbl = tree_table(raw_results)
tbl.to_excel(os.path.join(file_dir, "512k_x_1024_3.27886s.xlsx"), index=False)
print(tbl.head())

function  tree  Selecting Bootstrapped Samples  \
0            1                        0.054981   
1            2                        0.055047   
2            3                        0.053688   
3            4                        0.052345   
4            5                        0.052400   

function  Initialization of FindBestCondOblique  SampleProjection  \
0                                      0.002494          0.000009   
1                                      0.002365          0.000003   
2                                      0.002349          0.000003   
3                                      0.002344          0.000003   
4                                      0.002855          0.000006   

function  ApplyProjection  Bucket Allocation & Initialization=0  \
0                0.003391                              0.001848   
1                0.003408                              0.001835   
2                0.003403                              0.001843   
3               